### Yield From - Closing and Return

#### Closing the subgenerator

Essentially, the delegator code is "paused" at the "yield from subgen()" as long as subgen is not closed

When the subgen closes, delegator resumes running exactly where it was paused

If we close the delegator, (ie do d.close() )...
- then the subgenerator gets closed (if there is one active)
- and the delegator immediately gets closed as well

#### Returning from a generator

A generator can return:
- StopIteration

The returned value is embedded in the StopIteration exception
- we can extract that value:

In [ ]:
try:
    next(g)
except StopIteration as ex:
    print(ex.value)

And Python can do that to!

yield from is an expression
- it evaluates to the returned value of the subgenerator

In [ ]:
result = yield from subgen()

#### Code Examples

In [1]:
def subgen():
    try:
        while True:
            received = yield
            print(received)
    finally:
        print('subgen: closing...')

In [2]:
def delegator():
    s = subgen()
    yield from s
    yield 'delegator: subgen closed'
    print('delegator: closing...')

In [3]:
d = delegator()
next(d)

In [4]:
from inspect import getgeneratorstate, getgeneratorlocals

In [5]:
getgeneratorlocals(d)

{'s': <generator object subgen at 0x00000221504512C8>}

In [6]:
s = getgeneratorlocals(d)['s']

In [7]:
print(getgeneratorstate(d))
print(getgeneratorstate(s))

GEN_SUSPENDED
GEN_SUSPENDED


In [8]:
d.send('hello')

hello


In [9]:
s.close()

subgen: closing...


In [10]:
getgeneratorstate(s)

'GEN_CLOSED'

In [11]:
getgeneratorstate(d)

'GEN_SUSPENDED'

In [12]:
next(d)

'delegator: subgen closed'

In [13]:
next(d)

delegator: closing...


StopIteration: 

In [14]:
d = delegator()
next(d)
s = getgeneratorlocals(d)['s']
print(getgeneratorstate(d))
print(getgeneratorstate(s))

GEN_SUSPENDED
GEN_SUSPENDED


In [15]:
d.close()

subgen: closing...


In [16]:
print(getgeneratorstate(d))
print(getgeneratorstate(s))

GEN_CLOSED
GEN_CLOSED


In [17]:
def subgen():
    try:
        while True:
            received = yield
            print(received)
    finally:
        print('subgen: closing...')
        return 'subgen: return value'

In [18]:
s = subgen()

In [19]:
next(s)

In [20]:
s.send('hello')

hello


In [21]:
s.close()

subgen: closing...


In [22]:
s = subgen()
next(s)
s.send('hello')

hello


In [23]:
s.throw(GeneratorExit, 'force exit')

subgen: closing...


StopIteration: subgen: return value

In [24]:
s = subgen()
next(s)
try:
    s.throw(GeneratorExit, 'force exit')
except StopIteration as ex:
    print(ex.value)

subgen: closing...
subgen: return value


In [25]:
def subgen():
    yield 1
    yield 2
    return 'value'

In [27]:
s = subgen()

In [28]:
next(s)

1

In [29]:
next(s)

2

In [30]:
try:
    next(s)
except StopIteration as ex:
    print('return value:', ex.value)

return value: value


In [31]:
def subgen():
    try:
        yield 1
        yield 2
    finally:
        print('subgen closing...')
        return 100

In [32]:
def delegator():
    s = subgen()
    result = yield from s
    print('subgen returned:', result)
    yield 'delegator suspended'
    print('delegator closing')

In [33]:
d = delegator()

In [34]:
next(d)

1

In [35]:
next(d)

2

In [36]:
next(d)

subgen closing...
subgen returned: 100


'delegator suspended'